# Predicting Flights Delay: Flights Feature Engineering

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [4]:
flights = pd.read_csv('../data/flights_data_merge.csv')

In [5]:
original_flights =  pd.read_csv('../data/flights_data_merge.csv')

In [6]:
original_flights

Unnamed: 0     fl_date mkt_unique_carrier branded_code_share  \
0                0  2018-03-31                 UA                 UA   
1                1  2018-11-27                 DL                 DL   
2                2  2018-09-17                 AA       AA_CODESHARE   
3                3  2018-10-19                 G4                 G4   
4                4  2018-04-24                 DL       DL_CODESHARE   
...            ...         ...                ...                ...   
999995      499995  2019-02-17                 DL                 DL   
999996      499996  2019-07-09                 WN                 WN   
999997      499997  2019-08-12                 F9                 F9   
999998      499998  2019-10-09                 DL       DL_CODESHARE   
999999      499999  2019-10-23                 AS                 AS   

       mkt_carrier  mkt_carrier_fl_num op_unique_carrier tail_num  \
0               UA                2384                UA   N76288   
1               DL                1260                DL   N902DN   
2               AA                5906                YV   N912FJ   
3               G4                2272                G4    252NV   
4               DL                5583                EV   N738EV   
...            ...                 ...               ...      ...   
999995          DL                 468                DL   N835MH   
999996          WN                 368                WN   N7854B   
999997          F9                1759                F9   N356FR   
999998          DL                3968                OO   N8923A   
999999          AS                 475                AS   N273AK   

        op_carrier_fl_num  origin_airport_id  ... distance carrier_delay  \
0                    2384              11618  ...     1065           0.0   
1                    1260              11884  ...      546           NaN   
2                    5906              11298  ...      641           0.0   
3                    2272              10135  ...      518         654.0   
4                    5583              11042  ...      419           NaN   
...                   ...                ...  ...      ...           ...   
999995                468              12892  ...     2475           NaN   
999996                368              14831  ...      417           NaN   
999997               1759              11193  ...      773           NaN   
999998               3968              13487  ...      144           NaN   
999999                475              14747  ...      564           NaN   

        weather_delay nas_delay security_delay  late_aircraft_delay  \
0                 0.0       0.0            0.0                 38.0   
1                 NaN       NaN            NaN                  NaN   
2                 0.0       0.0            0.0                 47.0   
3                 0.0       0.0            0.0                  0.0   
4                 NaN       NaN            NaN                  NaN   
...               ...       ...            ...                  ...   
999995            NaN       NaN            NaN                  NaN   
999996            NaN       NaN            NaN                  NaN   
999997            NaN       NaN            NaN                  NaN   
999998            NaN       NaN            NaN                  NaN   
999999            NaN       NaN            NaN                  NaN   

        first_dep_time  total_add_gtime  longest_add_gtime  no_name  
0                  NaN              NaN                NaN      NaN  
1                  NaN              NaN                NaN      NaN  
2                  NaN              NaN                NaN      NaN  
3                  NaN              NaN                NaN      NaN  
4                  NaN              NaN                NaN      NaN  
...                ...              ...                ...      ...  
999995             NaN              NaN                NaN

In [7]:
def extract_features_df(df):
    
    '''
    Extract relevant features from flight dataframe.
        Input: Raw dataframe
        Output: Dataframe with relevant features for analysis
    '''
    
    new_df = df.drop(['branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num',
                      'op_carrier_fl_num', 'cancellation_code', 'carrier_delay', 'weather_delay', 
                      'nas_delay', 'security_delay', 'late_aircraft_delay', 'first_dep_time',
                      'total_add_gtime', 'longest_add_gtime', 'no_name', 'cancelled', 'diverted', 'taxi_out', 
                      'wheels_off', 'wheels_on', 'taxi_in', 'dup', 'dep_time', 'arr_time', 'actual_elapsed_time', 'air_time'], axis=1).dropna()
    
    new_df['fl_date'] = pd.to_datetime(new_df['fl_date'], format='%Y-%m-%d')

    return new_df

In [8]:
flights = extract_features_df(flights)
flights.drop('Unnamed: 0', axis=1, inplace=True)

In [9]:
flights.drop('flights', axis=1, inplace=True)

In [10]:
flights.head()

fl_date mkt_unique_carrier op_unique_carrier tail_num  origin_airport_id  \
0 2018-03-31                 UA                UA   N76288              11618   
1 2018-11-27                 DL                DL   N902DN              11884   
2 2018-09-17                 AA                YV   N912FJ              11298   
3 2018-10-19                 G4                G4    252NV              10135   
4 2018-04-24                 DL                EV   N738EV              11042   

  origin                origin_city_name  dest_airport_id dest  \
0    EWR                      Newark, NJ            11697  FLL   
1    GEG                     Spokane, WA            14869  SLC   
2    DFW           Dallas/Fort Worth, TX            15624  VPS   
3    ABE  Allentown/Bethlehem/Easton, PA            13577  MYR   
4    CLE                   Cleveland, OH            12953  LGA   

        dest_city_name  crs_dep_time  dep_delay  crs_arr_time  arr_delay  \
0  Fort Lauderdale, FL          1705       46.0          2017       38.0   
1   Salt Lake City, UT           544        1.0           847      -10.0   
2       Valparaiso, FL          1650       54.0          1846       47.0   
3     Myrtle Beach, SC          1108      656.0          1239      654.0   
4         New York, NY           912      -10.0          1048       -8.0   

   crs_elapsed_time  distance  
0             192.0      1065  
1             123.0       546  
2             116.0       641  
3              91.0       518  
4              96.0       419

In [11]:
flights.shape

(980258, 16)

### Feature 1: Analysis of Dates

<b> 1. Separate the date into months and days </b>

In [12]:
flights['month'] = flights['fl_date'].dt.month
flights['day'] = flights['fl_date'].dt.day
flights.head()

fl_date mkt_unique_carrier op_unique_carrier tail_num  origin_airport_id  \
0 2018-03-31                 UA                UA   N76288              11618   
1 2018-11-27                 DL                DL   N902DN              11884   
2 2018-09-17                 AA                YV   N912FJ              11298   
3 2018-10-19                 G4                G4    252NV              10135   
4 2018-04-24                 DL                EV   N738EV              11042   

  origin                origin_city_name  dest_airport_id dest  \
0    EWR                      Newark, NJ            11697  FLL   
1    GEG                     Spokane, WA            14869  SLC   
2    DFW           Dallas/Fort Worth, TX            15624  VPS   
3    ABE  Allentown/Bethlehem/Easton, PA            13577  MYR   
4    CLE                   Cleveland, OH            12953  LGA   

        dest_city_name  crs_dep_time  dep_delay  crs_arr_time  arr_delay  \
0  Fort Lauderdale, FL          1705       46.0          2017       38.0   
1   Salt Lake City, UT           544        1.0           847      -10.0   
2       Valparaiso, FL          1650       54.0          1846       47.0   
3     Myrtle Beach, SC          1108      656.0          1239      654.0   
4         New York, NY           912      -10.0          1048       -8.0   

   crs_elapsed_time  distance  month  day  
0             192.0      1065      3   31  
1             123.0       546     11   27  
2             116.0       641      9   17  
3              91.0       518     10   19  
4              96.0       419      4   24

<b> 2. Obtain median arrival and departure delays for each month; categorize dates into weekend and weekday </b>

In [13]:
monthly_delays = flights.groupby('month')[['arr_delay', 'dep_delay']].median().reset_index().rename(columns={'arr_delay': 'month_arr_delay_avg', 'dep_delay':'month_dep_delay_avg'})

In [14]:
flights = pd.merge(flights, monthly_delays, on='month', how='inner')

In [15]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'op_unique_carrier', 'tail_num',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'crs_dep_time', 'dep_delay', 'crs_arr_time',
       'arr_delay', 'crs_elapsed_time', 'distance', 'month', 'day',
       'month_arr_delay_avg', 'month_dep_delay_avg'],
      dtype='object')

In [16]:
def is_weekend(date):
    weekno = date.weekday()
    
    if weekno < 5:
        return 0
    else:
        return 1

In [17]:
flights['is_weekend'] = flights['fl_date'].map(is_weekend)

<b> 3. Convert time to datetime objects, separate into hours. </b>

In [18]:
def convert_time(time):
    
    if time == 2400:
        return '23:59'
    
    time_string = str(int(time))
    
    if len(time_string) < 4:
        for i in range(4-len(time_string)):
            time_string = '0' + time_string
    
    return time_string[:2] + ':' + time_string[2:]

In [19]:
flights['crs_dep_time'] = flights['crs_dep_time'].map(convert_time)
flights['crs_arr_time'] = flights['crs_arr_time'].map(convert_time)

In [20]:
flights['crs_dep_time'] = pd.to_datetime(flights['crs_dep_time'], format='%H:%M')
flights['crs_arr_time'] = pd.to_datetime(flights['crs_arr_time'], format='%H:%M')

In [21]:
flights['dep_hour'] = flights['crs_dep_time'].dt.hour
flights['dep_min'] = flights['crs_dep_time'].dt.minute

In [22]:
flights['arr_hour'] = flights['crs_arr_time'].dt.hour
flights['arr_min'] = flights['crs_arr_time'].dt.minute

In [23]:
flights.drop(['crs_dep_time', 'crs_arr_time'], axis=1, inplace=True)

<b> 4. Categorize arrival and departure times </b>

In [24]:
def get_time_desc(hour):
    if hour == 0:
        return 'midnight'
    elif hour > 0 and hour <= 4:
        return 'early morning'
    elif hour > 4 and hour <= 10:
        return 'morning'
    elif hour > 10 and hour <= 15:
        return 'noon'
    elif hour > 15 and hour <= 21:
        return 'evening'
    else:
        return 'late night'

In [25]:
flights['dep_time_desc'] = flights['dep_hour'].map(get_time_desc)
flights['arr_time_desc'] = flights['arr_hour'].map(get_time_desc)

### Feature 2: Tail Number Analysis

<b> 1. The average arrival delays amongst the different tail numbers are different, therefore they can be taken into account in a feature called plane quality. </b>

In [26]:
def get_plane_score(df):
    plane_quality = df.groupby('tail_num')[['arr_delay']].median().reset_index().rename(columns={'arr_delay': 'plane_score'})
    return pd.merge(df, plane_quality, on='tail_num', how='inner')

In [27]:
flights = get_plane_score(flights)

In [28]:
flights.shape

(980258, 26)

<b> 2. Get the number of flights each aircraft flies in that particular day. Aircrafts that fly a lot of times in one day can encounter more delays. </b>

In [29]:
num_flights_plane = flights.groupby(['fl_date', 'tail_num']).size().reset_index().rename(columns={0: 'num_flights_plane'})

In [30]:
num_flights_plane.sort_values(['fl_date', 'tail_num'])

fl_date tail_num  num_flights_plane
0      2018-01-01    220NV                  1
1      2018-01-01    228NV                  1
2      2018-01-01    230NV                  1
3      2018-01-01    231NV                  1
4      2018-01-01    232NV                  2
...           ...      ...                ...
862191 2019-12-31   N994AT                  1
862192 2019-12-31   N994NN                  1
862193 2019-12-31   N995JL                  1
862194 2019-12-31   N998AT                  1
862195 2019-12-31   N998NN                  1

[862196 rows x 3 columns]

In [31]:
flights = pd.merge(flights, num_flights_plane, on=['fl_date', 'tail_num'], how='inner')

### Feature 3: Destination and Arrival Airport Analysis

<b> 1. Get airport busyness. It can be hypothesized that busy airports with a lot of flights can face more delays. </b>

In [32]:
high_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[0:6]['origin'].tolist()
medhigh_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[6:25]['origin'].tolist()
medium_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[25:117]['origin'].tolist()
low_busy_dep = flights.groupby('origin').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[117:]['origin'].tolist()

def get_origin_busy(code):
    if code in high_busy_dep:
        return 'high'
    elif code in medhigh_busy_dep:
        return 'medium-high'
    elif code in medium_busy_dep:
        return 'medium'
    else:
        return 'low'
    
high_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[0:6]['dest'].tolist()
medhigh_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[6:25]['dest'].tolist()
medium_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[25:118]['dest'].tolist()
low_busy_dest = flights.groupby('dest').size().reset_index().rename(columns={0: 'num_flights'}).sort_values(by='num_flights', ascending=False)[118:]['dest'].tolist()

def get_dest_busy(code):
    if code in high_busy_dest:
        return 'high'
    elif code in medhigh_busy_dest:
        return 'medium-high'
    elif code in medium_busy_dest:
        return 'medium'
    else:
        return 'low'


In [33]:
flights['orr_busy'] = flights['origin'].map(get_origin_busy)
flights['arr_busy'] = flights['dest'].map(get_dest_busy)

<b> 2. Analyze Routes! Let's take a look at a combination of airline code, origin, destination, and month and obtain the following features: median, mean, 25%, and 75% of arrival delays. </b>

In [34]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'op_unique_carrier', 'tail_num',
       'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id',
       'dest', 'dest_city_name', 'dep_delay', 'arr_delay', 'crs_elapsed_time',
       'distance', 'month', 'day', 'month_arr_delay_avg',
       'month_dep_delay_avg', 'is_weekend', 'dep_hour', 'dep_min', 'arr_hour',
       'arr_min', 'dep_time_desc', 'arr_time_desc', 'plane_score',
       'num_flights_plane', 'orr_busy', 'arr_busy'],
      dtype='object')

In [35]:
flights['route'] = flights['op_unique_carrier'] + flights['origin'] + flights['dest'] + flights['month'].apply(lambda x: str(x))

In [36]:
route_mean = flights.groupby('route')[['arr_delay']].mean().reset_index().rename(columns={'arr_delay': 'route_mean_delay'})
route_median = flights.groupby('route')[['arr_delay']].median().reset_index().rename(columns={'arr_delay': 'route_median_delay'})
route_lq = flights.groupby('route')[['arr_delay']].quantile(0.25).reset_index().rename(columns={'arr_delay': 'route_lq_delay'})
route_uq = flights.groupby('route')[['arr_delay']].quantile(0.75).reset_index().rename(columns={'arr_delay': 'route_uq_delay'})

In [37]:
route_info = pd.merge(route_mean, route_median)
route_info = pd.merge(route_info, route_lq)
route_info = pd.merge(route_info, route_uq)

In [38]:
flights.shape

(980258, 30)

In [39]:
flights = pd.merge(flights, route_info, on='route', how='inner')

In [40]:
flights.shape

(980258, 34)

In [41]:
flights.head()

fl_date mkt_unique_carrier op_unique_carrier tail_num  origin_airport_id  \
0 2018-03-31                 UA                UA   N76288              11618   
1 2019-03-07                 UA                UA   N37468              11618   
2 2018-03-02                 UA                UA   N68823              11618   
3 2019-03-20                 UA                UA   N68823              11618   
4 2019-03-11                 UA                UA   N39450              11618   

  origin origin_city_name  dest_airport_id dest       dest_city_name  ...  \
0    EWR       Newark, NJ            11697  FLL  Fort Lauderdale, FL  ...   
1    EWR       Newark, NJ            11697  FLL  Fort Lauderdale, FL  ...   
2    EWR       Newark, NJ            11697  FLL  Fort Lauderdale, FL  ...   
3    EWR       Newark, NJ            11697  FLL  Fort Lauderdale, FL  ...   
4    EWR       Newark, NJ            11697  FLL  Fort Lauderdale, FL  ...   

   arr_time_desc  plane_score  num_flights_plane     orr_busy     arr_busy  \
0        evening         -7.0                  2  medium-high  medium-high   
1     late night         -8.0                  1  medium-high  medium-high   
2        evening         -8.0                  1  medium-high  medium-high   
3           noon         -8.0                  2  medium-high  medium-high   
4     late night         -5.0                  1  medium-high  medium-high   

       route  route_mean_delay  route_median_delay  route_lq_delay  \
0  UAEWRFLL3              0.24               -11.0           -18.0   
1  UAEWRFLL3              0.24               -11.0           -18.0   
2  UAEWRFLL3              0.24               -11.0           -18.0   
3  UAEWRFLL3              0.24               -11.0           -18.0   
4  UAEWRFLL3              0.24               -11.0           -18.0   

   route_uq_delay  
0             4.0  
1             4.0  
2             4.0  
3             4.0  
4             4.0  

[5 rows x 34 columns]

<b> 3. Let's obtain the probability of getting a delay greater than 10 minutes out of all the total flights. </b>

In [42]:
delayed_flights = flights[flights['arr_delay'] >= 10].groupby('route').size().reset_index().rename(columns={0: 'num_delayed_routes'})

In [43]:
total_flights = flights.groupby('route').size().reset_index().rename(columns={0: 'total_routes'})

In [44]:
all_flights = pd.merge(total_flights, delayed_flights, on='route', how='left')

In [45]:
all_flights['route_prob_delay'] = all_flights['num_delayed_routes']/all_flights['total_routes']

In [46]:
all_flights.fillna(0, inplace=True)

In [47]:
all_flights.head()

route  total_routes  num_delayed_routes  route_prob_delay
0   9EABEATL1             3                 0.0          0.000000
1  9EABEATL10             6                 1.0          0.166667
2  9EABEATL11             8                 3.0          0.375000
3  9EABEATL12             6                 2.0          0.333333
4   9EABEATL2             3                 1.0          0.333333

In [48]:
flights = pd.merge(flights, all_flights, on='route', how='inner')

<b> 4. Get the mean, median, 25% and 75% of historical departure delays for each route! </b>

In [49]:
route_mean_dep = flights.groupby('route')[['dep_delay']].mean().reset_index().rename(columns={'dep_delay': 'route_mean_dep_delay'})
route_median_dep = flights.groupby('route')[['dep_delay']].median().reset_index().rename(columns={'dep_delay': 'route_median_dep_delay'})
route_lq_dep = flights.groupby('route')[['dep_delay']].quantile(0.25).reset_index().rename(columns={'dep_delay': 'route_lq_dep_delay'})
route_uq_dep = flights.groupby('route')[['dep_delay']].quantile(0.75).reset_index().rename(columns={'dep_delay': 'route_uq_dep_delay'})

In [50]:
route_info_dep = pd.merge(route_mean_dep, route_median_dep)
route_info_dep = pd.merge(route_info_dep, route_lq_dep)
route_info_dep = pd.merge(route_info_dep, route_uq_dep)

In [51]:
flights = pd.merge(flights, route_info_dep, on='route', how='inner')

In [52]:
flights.shape

(980258, 41)

# That's all the features so far! Let's export the file to a csv!

In [53]:
#flights.to_csv('../data/data-feature/flights-feature-edit.csv')